<a href="https://colab.research.google.com/github/snvssk/data298A/blob/saidev/Project_XgBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

Dataset Import

In [ ]:
dataset = pd.read_csv('./all_combined_city.csv')

### Input and Output Values

In [ ]:
dataset['GHI'].isnull().values.sum()

0

In [ ]:
target = dataset['GHI']#keeping only target(ghi) data here
target

0          0
1          0
2          0
3          0
4          0
        ... 
33484    332
33485    296
33486    230
33487    160
33488     88
Name: GHI, Length: 33489, dtype: int64

In [ ]:
input= dataset.drop(['GHI'], axis=1)
input

,Year,Month,Day,Hour,Cloud_Type,Dew_Point,Solar_Zenith_Angle,Surface_Albedo,Wind_Speed,Precipitable_Water,Wind_Direction,Relative_Humidity,Temperature,Pressure
0,2016,1,1,0.0,1,0,165.44,0.099,1.0,0.705,72.0,87.98,4.0,1020.0
1,2016,1,1,0.5,1,0,165.13,0.099,1.0,0.714,72.0,87.97,3.0,1020.0
2,2016,1,1,1.0,1,0,162.29,0.099,1.0,0.723,73.1,87.91,3.0,1020.0
3,2016,1,1,1.5,1,0,157.90,0.099,1.0,0.730,73.1,87.91,3.0,1020.0
4,2016,1,1,2.0,1,0,152.70,0.099,1.0,0.737,65.4,87.69,3.0,1020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33484,2017,11,28,14.0,8,7,65.60,0.104,2.9,1.954,332.7,59.20,16.0,1020.0
33485,2017,11,28,14.5,8,7,69.02,0.104,2.7,1.991,332.7,63.14,16.0,1020.0
33486,2017,11,28,15.0,8,7,72.95,0.104,2.6,2.028,336.1,62.80,16.0,1020.0
33487,2017,11,28,15.5,8,7,77.32,0.104,1.9,2.031,336.1,67.01,15.0,1020.0


Feature Scaling

In [ ]:
#Normalize the dataset
sc = StandardScaler()
input_normalized = sc.fit_transform(input)

Split Data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(input_normalized, target, test_size = 0.2, random_state = 0)

Training XGBoost on the Training set

In [ ]:
from sklearn.datasets import make_classification
from xgboost import XGBClassifier
model = XGBClassifier(
    learning_rate=0.02,
    n_estimators=10,
    objective="binary:logistic",
    nthread=3,
    tree_method="gpu_hist"  # this enables GPU.
)

# fit the model
model.fit(X_train, y_train)

XGBClassifier(learning_rate=0.02, n_estimators=10, nthread=3,
              objective='multi:softprob', tree_method='gpu_hist')

Confustion Matrix 


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[3386    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]]


0.5140340400119439

In [ ]:
import math

MSE = np.square(np.subtract(y_test, y_pred)).mean() 
 
RMSE = math.sqrt(MSE)

In [ ]:
MSE

8894.313526425798

In [ ]:
# Considering RMSE as this is regression problem
RMSE

94.30966825530561

In [ ]:
# Calculating R2 score

from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
r2

0.9076422592991343

Need to Try GridSearchCV in Future. Below code is for future reference


In [ ]:
estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)

In [ ]:
parameters = {
    'max_depth': range (1, 2, 5),
    'n_estimators': range(10, 30, 100),
    'learning_rate': [0.1, 0.01, 0.05]
}

In [ ]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'neg_mean_squared_error',
    n_jobs = None,
    cv = 3,
    verbose=True
)

In [ ]:
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)
print("Lowest RMSE: ", (-grid_search.best_score_)**(1/2.0))

Fitting 3 folds for each of 3 candidates, totalling 9 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  UserWarning,


Best parameters: {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 10}
Lowest RMSE:  151.7231560416114
